<img align="right" width="200" height="200" src="https://static.tildacdn.com/tild6236-6337-4339-b337-313363643735/new_logo.png">

# Spark Structured Streaming III
**Андрей Титов**  
tenke.iu8@gmail.com  

## На этом занятии
+ Foreach Batch Sink
+ FAIR Scheduler

## Foreach Batch Sink

### В чем проблема обычных "синков" ?

Подготовим вспомогательные функции

In [ ]:
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.DataFrame

def createConsoleSink(chkName: String, df: DataFrame) = {
    df
    .writeStream
    .format("console")
    .trigger(Trigger.ProcessingTime("10 seconds"))
    .option("checkpointLocation", s"chk/$chkName")
    .option("truncate", "false")
    .option("numRows", "20")
}

In [ ]:
import org.apache.spark.sql.SparkSession

def killAll() = {
    SparkSession
        .active
        .streams
        .active
        .foreach { x =>
                    val desc = x.lastProgress.sources.head.description
                    x.stop
                    println(s"Stopped ${desc}")
        }               
}

In [ ]:
import org.apache.spark.sql.functions._

def airportsDf() = {
    val csvOptions = Map("header" -> "true", "inferSchema" -> "true")
    spark.read.options(csvOptions).csv("datasets/airport-codes.csv")
}

def randomIdent() = {
    
    val idents = airportsDf().select('ident).limit(20).distinct.as[String].collect

    val columnArray = idents.map( x => lit(x) )
    val sparkArray = array(columnArray:_*)
    val shuffledArray = shuffle(sparkArray)

    shuffledArray(0)
}

Удалим старые чекпоинты

In [ ]:
import sys.process._
"rm -rf chk".!!

Создадим и запустим два стрима на основе одного датафрейма `myStream`

In [ ]:
val myStream = spark
    .readStream
    .format("rate")
    .load
    .withColumn("ident", randomIdent())

In [ ]:
createConsoleSink("state1", myStream).start

In [ ]:
createConsoleSink("state1", myStream).start

При запуске второго стрима мы получаем ошибку, связанную с попыткой переиспользования активного чекпоинта. Запустим второй стрим с новым чекпоинтом:

In [ ]:
createConsoleSink("state2", myStream).start

Данный эксперимент показывает, что два стрима, даже если созданы на основе одного датафрейма, не могут использовать общий чекпоинт, а значит:
- они работают независимо
- чтение данных из источника происходит у каждого стрима отдельно
- один стрим может начать отставать (лагать) от другого стрима

Остановим запущенные стримы

In [ ]:
killAll

**foreachBatch** синк - это синк, позволяющий применять **произвольную** функцию к каждому батчу в стриме, работая с ним, как со **статическим** датафреймом:

In [ ]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.streaming.Trigger

def createSink(chkName: String, df: DataFrame)(batchFunc: (DataFrame, Long) => Unit) = {
    df
    .writeStream
    .trigger(Trigger.ProcessingTime("10 seconds"))
    .option("checkpointLocation", s"chk/$chkName")
    .foreachBatch(batchFunc)
}

In [ ]:
createSink("state4", myStream) { (df, id) => 
    df.show(1, false)
    println(s"This is batch $id")
}.start

In [ ]:
killAll

### Выводы:
- Обычные синки позволяют создавать простые пайплайны; в реальной жизни чаще используется `foreachBatch`
- `foreachBatch` позволяет:
  + Можно использовать любое API внутри функции
  + Можно использовать `cache()` и `persist()`
  + Можно выполнять запись в несколько разных мест
  + Поддерживает режимы `append`, `update`, `complete`
  + Использовать `batch id`

## FAIR Scheduler
До этого момента мы всегда action'ы над датафреймами один из другим, то есть последовательно:

In [ ]:
val df = spark.range(0,100)
df.count
df.show
df.collect

Данный подход имеет существенный недостаток - низкая утилизация ресурсов, предоставленных Spark приложению, поскольку каждое действие блокирует основной поток на драйвере и не позволяет выполняться следующим, даже если у приложения еще есть свободные ресурсы.

Вышеописанная проблема не является критичной для обычных ETL приложений, однако в стримах, где задержка обработки данных является одним из ключевых параметров, простой выделенных ресурсов недопустим.

In [ ]:
import org.apache.spark.sql.functions._

val delay = udf { () => Thread.sleep(1000); true}

In [ ]:
val df = spark.range(0,30, 1, 1)

In [ ]:
df.withColumn("foo", delay()).collect

Если запустить вычисление данного датафрейма несколько раз, то он будет выполняться последовательно:

In [ ]:
spark.time { 
    1 to 5 foreach { _ => 
        val df = spark.range(0, 10, 1, 1)
        df.withColumn("foo", delay()).collect
    }
}

Используя метод `par`, мы можем запустить вычисление на драйвере одновременно для всех датафреймов, однако планировщик Spark приложения работает в режиме FIFO, поэтому на воркерах партиции разных датафреймов будут все еще обрабатываться последовательно

In [ ]:
spark.time { 
    (1 to 5).par.foreach { _ => 
        val df = spark.range(0, 10, 1, 1)
        df.withColumn("foo", delay()).collect
    }
}

Однако, если переключить режим планировщика в `FAIR`, то все 5 действий будут выполняться параллельно. Для этого необходимо установить две опции:
```
spark.scheduler.mode FAIR
spark.scheduler.allocation.file /path/to/fairscheduler.xml
```
Файл `fairscheduler.xml` должен содержать:
```xml
<?xml version="1.0"?>
<allocations>
  <pool name="default">
    <schedulingMode>FAIR</schedulingMode>
    <weight>1</weight>
  </pool>
</allocations>
```

Выполнив перезапуск Spark приложения с новыми параметрами (в этом месте необходимо перезапустить kernel или весь jupyter), можно убедиться, что код ниже отработает гораздо быстрее:

In [ ]:
spark.time { 
    (1 to 5).par.foreach { _ => 
        val df = spark.range(0, 10, 1, 1)
        df.withColumn("foo", delay()).collect
    }
}

### Выводы:
- Spark позволяет запускать действия параллельно
- Для параллельного запуска действий должны быть выполнены следующие условия:
  + Планировщик Spark должен быть переведен в FAIR режим
  + Пул `default` (должен быть переведен в FAIR режим)
  + Действия должны запускаться на драйвере паралелльно (можно использовать любое API, поддерживающее multithreading)

In [ ]:
spark.stop()